# ===========================================================
# Full pipeline for the influence matrix estimation problem on the supervised dataset from the Jeopardy-like logs for comparison on different models
# ===========================================================

Goals:
1. Split the data into test and train, and validation for multiple runs
2. Formulate all different models of convex optimization, neural networks, and tower models.
3. Give the same splits to all models, tune the hyperparameters with validation set, and report train and test erros as a pickle, a table, and a figure

#### Started on: 30 Dec 2019
#### Last update: 02 Jan 2020

# Imports

In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals

import imp
import sys
import scipy as sp
import cvxpy as cp
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv1D, LSTM, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Concatenate, Reshape, Embedding, Dot
from tensorflow.keras.models import Model
sys.path.insert(0, '../src/')
%matplotlib inline

import utils
from mytimer import Timer

/home/omid/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Parameters

In [2]:
DATA_FILE_PATH = '/home/omid/Datasets/Jeopardy/supervised_data.pk'
# DATA_FILE_PATH = '/home/omid/Datasets/Jeopardy/supervised_data_roberta.pk'
TEST_FRACTION = 0.2
RUNS = 2

# Helper functions

## General functions 

In [3]:
def reload():
    imp.reload(utils)

In [4]:
def my_init(shape, dtype=None):
    return np.ones(shape) * 0.25

In [5]:
# def get_eigvec_of_laplacian(A: np.matrix) -> np.matrix:
# #     D = np.diag(np.array(np.sum(A, axis=0))[0])
# #     L = D - A
# #     return np.matrix(np.linalg.eig(L)[1])
#     n, m = A.shape
#     diags = A.sum(axis=1).flatten()
#     D = sp.sparse.spdiags(diags, [0], m, n, format='csr')
#     L = D - A
#     with sp.errstate(divide='ignore'):
#         diags_sqrt = 1.0/sp.sqrt(diags)
#     diags_sqrt[sp.isinf(diags_sqrt)] = 0
#     DH = sp.sparse.spdiags(diags_sqrt, [0], m, n, format='csr')
#     DH = DH.todense()
#     normalized_L = DH.dot(L.dot(DH))
#     return normalized_L

## Models

### Boilerplate model functions

In [6]:
def comput_error(y_train_or_validation_or_test_true, y_train_or_validation_or_test_predicted, estimation_name, error_type_str):
    """Computes the error."""
    err = 0
    for index in range(len(y_train_or_validation_or_test_true)):
        groundtruth = y_train_or_validation_or_test_true[index][estimation_name]
        predicted = y_train_or_validation_or_test_predicted[index]
        # TODO(@Omid): CHECK WHETHER THIS IS JUST AN INT THEN FIND THE MOST INFLUENTIAL PERSON FROM THE ESTIAMTED MATRIX.
        err += utils.matrix_estimation_error(
            true_matrix=groundtruth, pred_matrix=predicted, type_str=error_type_str)
    err /= len(y_train_or_validation_or_test_true)
    return err


def model_builder(
        X_train,
        y_train,
        X_test,
        y_test,
        feature_names,
        estimation_name='influence_matrix',
        error_type_str='normalized_frob_norm',
        tune_hyperparameters_by_validation=True,
        with_replication=True,
        lambdas = [0, 0.1, 1, 10, 100, 1000],
        model_func=convex_optimization_model_func,
        params={'with_constraints': True, 'n_splits': 3}):
    
    # For the baseline models.
    if model_func == 'average':
        mats = []
        for i in range(len(y_train)):
            mats.append(y_train[i][estimation_name])
        y_baseline_predicted = [np.matrix(np.mean(mats, axis=0)) for _ in range(len(y_train))]
    elif model_func == 'uniform':
        y_baseline_predicted = [np.matrix(np.ones((4, 4)) * 0.25) for _ in range(len(y_train))]
    elif model_func == 'random':
        y_baseline_predicted = [np.matrix(
            utils.make_matrix_row_stochastic(
                np.random.rand(4, 4))) for _ in range(len(y_train))]
    if model_func in ['average', 'uniform', 'random']:
        y_train_average = []
        train_error = comput_error(
            y_train, y_baseline_predicted, estimation_name=estimation_name, error_type_str=error_type_str)
        test_error = comput_error(
            y_test, y_baseline_predicted, estimation_name=estimation_name, error_type_str=error_type_str)
        return train_error, test_error, _
    
    # For the proposed models.
    validation_errors = defaultdict(lambda: 0)
    if tune_hyperparameters_by_validation:
        print('{}-fold validation ...'.format(params['n_splits']))
        kf = KFold(n_splits=params['n_splits'])
        for train_index, validation_index in kf.split(X_train):
            X_train_subset, X_validation = X_train[train_index], X_train[validation_index]
            y_train_subset, y_validation = y_train[train_index], y_train[validation_index]
            if with_replication:
                X_train_subset, y_train_subset = utils.replicate_matrices_in_train_dataset_with_reordering(
                    X_train_subset, y_train_subset)
                X_train_subset = np.array(X_train_subset)
                y_train_subset = np.array(y_train_subset)
            print('Shapes of train: {}, validation: {}, test: {}.'.format(
                X_train_subset.shape, X_validation.shape, X_test.shape))
            for lambdaa in lambdas:
                validation_errors[lambdaa] += model_func(
                    X_train=X_train_subset,
                    y_train=y_train_subset,
                    X_validation_or_test=X_validation,
                    y_validation_or_test=y_validation,
                    feature_names=feature_names,
                    estimation_name=estimation_name,
                    lambdaa=lambdaa,
                    error_type_str=error_type_str,
                    params=params)[1]
        best_lambda = min(validation_errors, key=validation_errors.get)
    else:
        best_lambda = 0.1
    print('Training with the best lambda: {} on entire training set...'.format(best_lambda))
    train_error, test_error = model_func(
        X_train=X_train,
        y_train=y_train,
        X_validation_or_test=X_test,
        y_validation_or_test=y_test,
        feature_names=feature_names,
        estimation_name=estimation_name,
        lambdaa=best_lambda,
        error_type_str=error_type_str,
        params=params)
    return train_error, test_error, validation_errors

### Specific model functions

In [19]:
def convex_optimization_model_func(
        X_train,
        y_train,
        X_validation_or_test,
        y_validation_or_test,
        feature_names,
        estimation_name,
        lambdaa,
        error_type_str,
        params={'with_constraints': True}):
    
    def predict(data_element, feature_names, B, Ws, is_solved=False):
        """Defines the prediction function."""
        predicted = 0
        if is_solved:
            predicted += B.value
        else:
            predicted += B
        for feature_name in feature_names:
            if is_solved:
                W_for_this_feature = Ws[feature_name].value
            else:
                W_for_this_feature = Ws[feature_name]
            if len(data_element[feature_name].shape) == 1:
                # If it was a vector, it makes a matrix with it.
                p = data_element[feature_name]
                data_element_matrix = np.row_stack([p, p, p, p])
            else:
                # Unless it is already a matrix.
                data_element_matrix = data_element[feature_name]
            predicted += (W_for_this_feature * data_element_matrix)
        return predicted
    
    def predict_all_after_solving(X_train_or_validation_or_test, B, Ws, feature_names):
        """Predicts for all data points in the given set."""
        return [
            predict(
                data_element=data_element,
                feature_names=feature_names,
                B=B,
                Ws=Ws,
                is_solved=True) 
            for data_element in X_train_or_validation_or_test]

    # Creating variables.
    Ws = {}
    for feature_name in feature_names:
        if len(X_train[0][feature_name].shape) == 1:
            # If it was a vector, it makes a matrix with it.
            Ws[feature_name] = cp.Variable(
                len(X_train[0][feature_name]), len(X_train[0][feature_name]))
        else:
            # Unless it is already a matrix.
            Ws[feature_name] = cp.Variable(
                X_train[0][feature_name].shape[0], X_train[0][feature_name].shape[1])
    B = cp.Variable(4, 4)

    # Computing loss.
    constraints = []
    losses = 0
    for index in range(len(X_train)):
        element = X_train[index]
        estimation_groundtruth = y_train[index][estimation_name]

        # Defining the estimation function.
        estimation_predicted = predict(
            data_element=element, feature_names=feature_names, B=B, Ws=Ws, is_solved=False)

        # Defining the loss function.
        loss = cp.sum_squares(estimation_predicted - estimation_groundtruth)

        losses += loss
        if params['with_constraints']:
            constraints += [estimation_predicted >= 0]
            constraints += [cp.sum_entries(estimation_predicted, axis=1) == 1]

    # Computing regularization.
    regluarization = cp.norm1(B)
    for feature_name in feature_names:
        regluarization += cp.norm1(Ws[feature_name])

    # Solving the convex problem.
    objective = cp.Minimize(losses + lambdaa * regluarization)
    prob = cp.Problem(objective, constraints)
    result = prob.solve(solver=cp.MOSEK)
    print('The status of solution was: {} and the result was: {}'.format(prob.status, result))

    # Predicting and computing trian error.
    y_train_predicted = predict_all_after_solving(
        X_train_or_validation_or_test=X_train, B=B, Ws=Ws, feature_names=feature_names)
    train_error = comput_error(
        y_train_or_validation_or_test_true=y_train,
        y_train_or_validation_or_test_predicted=y_train_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)
    
    # Predicting and computing validation or test error.
    y_validation_or_test_predicted = predict_all_after_solving(
        X_train_or_validation_or_test=X_validation_or_test, B=B, Ws=Ws, feature_names=feature_names)
    validation_or_test_error = comput_error(
        y_train_or_validation_or_test_true=y_validation_or_test,
        y_train_or_validation_or_test_predicted=y_validation_or_test_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)
    
    return train_error, validation_or_test_error

In [30]:
def concatinated_deep_neural_network_model_func(
        X_train,
        y_train,
        X_validation_or_test,
        y_validation_or_test,
        feature_names,
        estimation_name,
        lambdaa,
        error_type_str,
        params={'n_epochs': 10, 'batch_size': 32}):
    
#     def predict(data_element, feature_names, B, Ws, is_solved=False):
#         """Defines the prediction function."""
#         return 0
    
#     def predict_all_after_solving(X_train_or_validation_or_test, B, Ws, feature_names):
#         """Predicts for all data points in the given set."""
#         return []

    flatten_X_train = []
    flatten_y_train = []
    for i in range(len(X_train)):
        features = X_train[i]
        label = y_train[i][estimation_name]            
        flatten_X_train.append(np.hstack(
            [np.array(features[feature_name].flatten())[0] for feature_name in feature_names]))
        flatten_y_train.append(np.array(label.flatten())[0])
    flatten_X_train = np.array(flatten_X_train)
    flatten_y_train = np.array(flatten_y_train)

    flatten_X_validation_or_test = []
    flatten_y_validation_or_test = []
    for i in range(len(X_validation_or_test)):
        features = X_validation_or_test[i]
        label = y_validation_or_test[i][estimation_name]
        flatten_X_validation_or_test.append(np.hstack(
            [np.array(features[feature_name].flatten())[0] for feature_name in feature_names]))
        flatten_y_validation_or_test.append(np.array(label.flatten())[0])
    flatten_X_validation_or_test = np.array(flatten_X_validation_or_test)
    flatten_y_validation_or_test = np.array(flatten_y_validation_or_test)
                              
    _, input_size = flatten_X_train.shape
    print('Input size for the neural network was: {}'.format(input_size))

    model = Sequential([
        Dense(
            units=32,
            kernel_initializer='he_normal',
            activation='relu',
            input_shape=(input_size,),
            kernel_regularizer=regularizers.l1(lambdaa),
            activity_regularizer=regularizers.l1(lambdaa)),
#         Dropout(0.5),
#         Dense(
#             units=64,
#             kernel_initializer='he_normal',
#             activation='relu',
#             kernel_regularizer=regularizers.l1(lambdaa),
#             activity_regularizer=regularizers.l1(lambdaa)),
#         Dropout(0.5),
#         Dense(
#             units=32,
#             kernel_initializer='he_normal',
#             activation='relu',
#             kernel_regularizer=regularizers.l1(lambdaa),
#             activity_regularizer=regularizers.l1(lambdaa)),
#         Dropout(0.5),
        Dense(16, kernel_initializer=my_init, activation='softmax')])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.fit(flatten_X_train, flatten_y_train, epochs=params['n_epochs'], batch_size=params['batch_size'])

    # Predicting and computing trian error.
    y_train_predicted = [utils.make_matrix_row_stochastic(
        np.matrix(np.reshape(element, (4, 4)))) for element in model.predict(flatten_X_train)]
    train_error = comput_error(
        y_train_or_validation_or_test_true=y_train,
        y_train_or_validation_or_test_predicted=y_train_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)
                                            
    # Predicting and computing trian error.
    y_test_predicted = [utils.make_matrix_row_stochastic(
        np.matrix(np.reshape(element, (4, 4)))) for element in model.predict(flatten_X_validation_or_test)]
    validation_or_test_error = comput_error(
        y_train_or_validation_or_test_true=y_test,
        y_train_or_validation_or_test_predicted=y_test_predicted,
        estimation_name=estimation_name,
        error_type_str=error_type_str)

    return train_error, validation_or_test_error

# Loading the data

In [8]:
data = utils.load_it(DATA_FILE_PATH)
print(len(data['X']))

264


### Just to see how much headroom there is:

In [9]:
mats = []
for i in range(len(data['y'])):
    mats.append(data['y'][i]['influence_matrix'])
np.mean(mats, axis=0)

array([[0.25784649, 0.25013211, 0.25917224, 0.23284916],
       [0.21997335, 0.3244428 , 0.22187468, 0.23370918],
       [0.21050033, 0.25039752, 0.31083892, 0.22826324],
       [0.25180346, 0.23839655, 0.24293584, 0.26686416]])

In [10]:
np.mean(mats, axis=0)

array([[0.25784649, 0.25013211, 0.25917224, 0.23284916],
       [0.21997335, 0.3244428 , 0.22187468, 0.23370918],
       [0.21050033, 0.25039752, 0.31083892, 0.22826324],
       [0.25180346, 0.23839655, 0.24293584, 0.26686416]])

In [11]:
np.std(mats, axis=0)

array([[0.11677549, 0.08129962, 0.08962429, 0.07536874],
       [0.08929333, 0.16587926, 0.08335618, 0.10916787],
       [0.08963767, 0.12213777, 0.16865608, 0.09193362],
       [0.09597464, 0.06509444, 0.05915974, 0.08748726]])

#### Message: It seems the std is small, and average is close to 0.25 everywhere but main diagonal which is slightly larger (due to having selfish people).

# Main body

In [12]:
data['X'][0].keys()

dict_keys(['average_of_previous_influence_matrices', 'content_embedding_matrix', 'individual_performance_hardness_weighted', 'emotion_valence', 'emotion_dominance', 'first_influence_matrix', 'individual_performance', 'sentiment', 'emotion_arousal', 'reply_duration'])

In [13]:
LAMBDAS = [0, 0.1, 1, 10, 100, 1000]
WITH_REPLICATION = True
ERROR_TYPE_STRS = ['normalized_frob_norm', 'mse', 'neg_corr', 'cosine_dist']
TUNE_HYPERPARAMETERS_BY_VALIDATION = True
SELECTED_MODEL_FUNCS = ['average', 'uniform', 'random', convex_optimization_model_func, concatinated_deep_neural_network_model_func]
FEATURE_NAMES_SET = [['individual_performance'], ['first_influence_matrix'], ['reply_duration'], ['content_embedding_matrix'], ['average_of_previous_influence_matrices']]
ESTIMATION_NAMES = ['influence_matrix', 'most_influentials']

## Estimation comparison pipeline

In [14]:
with Timer():
    estimation_name = ESTIMATION_NAMES[0]
    error_type_str = ERROR_TYPE_STRS[0]
    
    train_errors_in_runs = defaultdict(list)
    test_errors_in_runs = defaultdict(list)
    validation_errors_in_runs = defaultdict(list)
    for run in range(RUNS):
        print('Run', run + 1, '=>>>')
        X_train, X_test, y_train, y_test = train_test_split(
            np.array(data['X']), np.array(data['y']), test_size=TEST_FRACTION)
        for feature_names in FEATURE_NAMES_SET:
            print('Features: ', feature_names, '...')
            for selected_model_func in SELECTED_MODEL_FUNCS:
                print('Model: ', selected_model_func, '...')
                with Timer():
                    train_error, test_error, validation_errors = model_builder(
                        X_train=X_train,
                        y_train=y_train,
                        X_test=X_test,
                        y_test=y_test,
                        feature_names=feature_names,
                        estimation_name=estimation_name,
                        error_type_str=error_type_str,
                        tune_hyperparameters_by_validation=TUNE_HYPERPARAMETERS_BY_VALIDATION,
                        with_replication=WITH_REPLICATION,
                        lambdas=LAMBDAS,
                        model_func=selected_model_func,
                        params={'with_constraints': True, 'n_splits': 3, 'n_epochs': 10, 'batch_size': 32})
                key_str = str(selected_model_func) + ':' + str(feature_names)
                train_errors_in_runs[key_str].append(train_error)
                test_errors_in_runs[key_str].append(test_error)
                validation_errors_in_runs[key_str].append(validation_errors)

Run 1 =>>>
Features:  ['individual_performance'] ...
Model:  average ...
It took 0.05 seconds.
Model:  uniform ...
It took 0.01 seconds.
Model:  random ...
It took 0.03 seconds.
Model:  <function convex_optimization_model_func at 0x7f4d855e48c8> ...
3-fold validation ...
Shapes of train: (3360,), validation: (71,), test: (53,).
The status of solution was: optimal and the result was: 583.1121941681871
The status of solution was: optimal and the result was: 583.512192508478
The status of solution was: optimal and the result was: 587.1121940821129
The status of solution was: optimal and the result was: 623.1121996097442
The status of solution was: optimal and the result was: 983.1121791699754
Shapes of train: (3384,), validation: (70,), test: (53,).
The status of solution was: optimal and the result was: 598.8717429761807
The status of solution was: optimal and the result was: 599.2717523963174
The status of solution was: optimal and the result was: 602.8717441264012
The status of solutio

The status of solution was: optimal and the result was: 539.1487446717288
The status of solution was: optimal and the result was: 899.1487402733535
Training with the best lambda: 10 on entire training set...
The status of solution was: optimal and the result was: 69.81939097583108
It took 2.89 minutes.
Features:  ['average_of_previous_influence_matrices'] ...
Model:  average ...
It took 0.06 seconds.
Model:  uniform ...
It took 0.01 seconds.
Model:  random ...
It took 0.02 seconds.
Model:  <function convex_optimization_model_func at 0x7f4d855e48c8> ...
3-fold validation ...
Shapes of train: (3360,), validation: (71,), test: (53,).
The status of solution was: optimal and the result was: 339.93614073315484
The status of solution was: optimal and the result was: 340.3361416384775
The status of solution was: optimal and the result was: 343.9361332164393
The status of solution was: optimal and the result was: 379.9361389258591
The status of solution was: optimal and the result was: 739.9361

In [15]:
for setting_str, errors in test_errors_in_runs.items():
    print(setting_str, '=>> ', np.mean(errors), '+-', np.std(errors))

average:['average_of_previous_influence_matrices'] =>>  0.34364980214151475 +- 0.010812181527379783
uniform:['average_of_previous_influence_matrices'] =>>  0.3495383349306685 +- 0.010645381369812412
<function convex_optimization_model_func at 0x7f4d855e48c8>:['average_of_previous_influence_matrices'] =>>  0.2442421771438104 +- 0.01317857215832012
average:['individual_performance'] =>>  0.34364980214151475 +- 0.010812181527379783
<function convex_optimization_model_func at 0x7f4d855e48c8>:['individual_performance'] =>>  0.3436467014985325 +- 0.010812216276520847
random:['first_influence_matrix'] =>>  0.6410311027354627 +- 0.016657903363658855
uniform:['first_influence_matrix'] =>>  0.3495383349306685 +- 0.010645381369812412
random:['average_of_previous_influence_matrices'] =>>  0.6289606832203576 +- 0.012499024868696584
uniform:['individual_performance'] =>>  0.3495383349306685 +- 0.010645381369812412
average:['first_influence_matrix'] =>>  0.34364980214151475 +- 0.010812181527379783
<f

In [23]:
# X_train, X_test, y_train, y_test = train_test_split(
#     np.array(data['X']), np.array(data['y']), test_size=TEST_FRACTION)
# estimation_name = ESTIMATION_NAMES[0]

In [35]:
# a, b, c = model_builder(
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test,
#     feature_names=['first_influence_matrix'],
#     estimation_name=estimation_name,
#     error_type_str=ERROR_TYPE_STR,
#     tune_hyperparameters_by_validation=True,
#     with_replication=WITH_REPLICATION,
#     lambdas=LAMBDAS,
#     model_func=concatinated_deep_neural_network_model_func,
#     params={'n_splits': 3, 'n_epochs': 10, 'batch_size': 32})

3-fold validation ...
Shapes of train: (3360,), validation: (71,), test: (53,).
Input size for the neural network was: 16
Train on 3360 samples
Epoch 1/10
3360/3360 [==============================] - 1s 153us/sample - loss: 0.7392 - accuracy: 0.0094
Epoch 2/10
3360/3360 [==============================] - 0s 44us/sample - loss: 0.7362 - accuracy: 0.0094
Epoch 3/10
3360/3360 [==============================] - 0s 33us/sample - loss: 0.7353 - accuracy: 0.0094
Epoch 4/10
3360/3360 [==============================] - 0s 32us/sample - loss: 0.7351 - accuracy: 0.0094
Epoch 5/10
3360/3360 [==============================] - 0s 32us/sample - loss: 0.7351 - accuracy: 0.0094
Epoch 6/10
3360/3360 [==============================] - 0s 32us/sample - loss: 0.7350 - accuracy: 0.0094
Epoch 7/10
3360/3360 [==============================] - 0s 31us/sample - loss: 0.7349 - accuracy: 0.0094
Epoch 8/10
3360/3360 [==============================] - 0s 32us/sample - loss: 0.7349 - accuracy: 0.0094
Epoch 9/10
3360

3384/3384 [==============================] - 0s 44us/sample - loss: 87.7238 - accuracy: 0.0124
Epoch 3/10
3384/3384 [==============================] - 0s 45us/sample - loss: 53.5002 - accuracy: 0.0124
Epoch 4/10
3384/3384 [==============================] - 0s 42us/sample - loss: 30.3800 - accuracy: 0.0124
Epoch 5/10
3384/3384 [==============================] - 0s 39us/sample - loss: 15.5570 - accuracy: 0.0124
Epoch 6/10
3384/3384 [==============================] - 0s 40us/sample - loss: 7.1421 - accuracy: 0.0124
Epoch 7/10
3384/3384 [==============================] - 0s 39us/sample - loss: 2.7150 - accuracy: 0.0124
Epoch 8/10
3384/3384 [==============================] - 0s 38us/sample - loss: 1.0535 - accuracy: 0.0124
Epoch 9/10
3384/3384 [==============================] - 0s 41us/sample - loss: 0.7985 - accuracy: 0.0124
Epoch 10/10
3384/3384 [==============================] - 0s 43us/sample - loss: 0.7980 - accuracy: 0.0124
Input size for the neural network was: 16
Train on 3384 sampl

3384/3384 [==============================] - 0s 42us/sample - loss: 2605.5542 - accuracy: 0.0102
Epoch 5/10
3384/3384 [==============================] - 0s 42us/sample - loss: 1258.2676 - accuracy: 0.0102
Epoch 6/10
3384/3384 [==============================] - 0s 40us/sample - loss: 463.4276 - accuracy: 0.0102
Epoch 7/10
3384/3384 [==============================] - 0s 40us/sample - loss: 109.5737 - accuracy: 0.0102
Epoch 8/10
3384/3384 [==============================] - 0s 40us/sample - loss: 10.0105 - accuracy: 0.0102
Epoch 9/10
3384/3384 [==============================] - 0s 38us/sample - loss: 6.8571 - accuracy: 0.0102
Epoch 10/10
3384/3384 [==============================] - 0s 40us/sample - loss: 6.8491 - accuracy: 0.0102
Training with the best lambda: 1 on entire training set...
Input size for the neural network was: 16
Train on 211 samples
Epoch 1/10
211/211 [==============================] - 0s 2ms/sample - loss: 149.5711 - accuracy: 0.0107
Epoch 2/10
211/211 [==================

In [36]:
# a

0.3242584311623669

In [37]:
# b

0.32196516153850907

In [38]:
# c

defaultdict(<function __main__.model_builder.<locals>.<lambda>>,
            {0: 1.1056997508973119,
             0.1: 0.9879060022903541,
             1: 0.9876533514632948,
             10: 0.9879012837935006,
             100: 0.9880853469198327})